In [10]:
%load_ext autoreload
%autoreload 2
import os
os.sys.path.insert(0, '..') 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from isic_dataset import ISIC_2018

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

torch.manual_seed(42);

isic_train = ISIC_2018()
isic_val = ISIC_2018(split='val')

Start downloading from: https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_GroundTruth.zip


ISIC2018_Task3_Training_GroundTruth.zip: 41.0kB [00:00, 265kB/s]                             


Start downloading from: https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip


ISIC2018_Task3_Validation_Input.zip: 53.3MB [00:01, 35.6MB/s]                            


Start downloading from: https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_GroundTruth.zip


ISIC2018_Task3_Validation_GroundTruth.zip: 8.19kB [00:00, 61.9kB/s]

MEL: 1113 (11.11%), NV: 6705 (66.95%), BCC: 514 (5.13%), AKIEC: 327 (3.27%), BKL: 1099 (10.97%), DF: 115 (1.15%), VASC: 142 (1.42%)
MEL: 21 (10.88%), NV: 123 (63.73%), BCC: 15 (7.77%), AKIEC: 8 (4.15%), BKL: 22 (11.4%), DF: 1 (0.52%), VASC: 3 (1.55%)


In [7]:
from skorch.callbacks import LRScheduler
from skorch.callbacks import Checkpoint

lrscheduler = LRScheduler(
    policy='StepLR', step_size=7, gamma=0.1)
checkpoint = Checkpoint(
    f_params='best_model.pt', monitor='valid_acc_best')

In [8]:
from isic_resnet import ISICResNet18
from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

from utils import pytorch_helper

device = pytorch_helper.get_device()

net = NeuralNetClassifier(
    ISICResNet18, 
    criterion=nn.CrossEntropyLoss,
    lr=0.001,
    batch_size=4,
    max_epochs=25,
    module__output_features=len(isic_train.img_labels.keys()),
    optimizer=optim.SGD,
    optimizer__momentum=0.9,
    iterator_train__shuffle=True,
    iterator_train__num_workers=2,
    iterator_valid__num_workers=2,
    train_split=predefined_split(isic_val),
    callbacks=[lrscheduler, checkpoint],
    # device= device,
    classes=[torch.tensor, torch.tensor],    
    device= 'mps',
)

In [ ]:
net.fit(isic_train, y=None);